This script does two things:

* splits each dataset into two, 'existing' facilities and 'proposed' facilities. Existing links with proposed changes are considered 'existing'.
* partitions regional datasets spatially using the 14-area system (named "boundary_1" to "boundary_14") created during the TM2 networks creation process. If a line falls into multiple boundaries, the line won't be broken; the complete line would be included in each boundary.

In [1]:
import os
import fiona
import geopandas as gpd

In [2]:
box_folder = 'C:\\Users\\{}\\Box\\DataViz Projects\\Spatial Analysis and Mapping\\Active Transportation Plan\\Data'.format(os.getenv('USERNAME'))
local_folder = 'C:\\Users\\{}\\Documents\\GitHub\\Spatial-Analysis-Mapping-Projects\\Project-Documentation\\Active-Transportation-Plan-Data-Development\\data'.format(os.getenv('USERNAME'))

# input
data_folder = os.path.join(box_folder, 'geojson')
boundary_folder = os.path.join(box_folder, 'helpers', 'split_boundaries')

data_list = ['actc_bike_network_epsg4326',
             'batc_bike_network_v2_epsg4326',         # regional
             'caltrans_d4_bike_network_epsg4326',     # regional
             'ccag_bike_network_epsg4326',
             'ccta_bike_network_epsg4326',
             'nvta_bike_network_epsg4326',
             'oakland_bike_network_epsg4326',
             'san_jose_bike_nw_epsg4326',
             'scta_bike_network_epsg4326',
             'sfcta_bike_network_epsg4326',
             'sta_bike_network_epsg4326',
             'tam_bike_network_epsg4326',
             'vta_bike_network_v2_epsg4326'  # regional
]

# export folder
existing_folder = os.path.join(local_folder, 'geojson', 'existing')
proposed_folder = os.path.join(local_folder, 'geojson', 'proposed')

In [3]:
def data_partition(gdf_regional, gdf_regional_file, link_type, output_folder):
    
    for i in range(14):
        # read boundary file
        file = os.path.join(boundary_folder, 'boundary_'+str(i+1)+'.geojson')
        print('use {} to intersect'.format('boundary_'+str(i+1)))
        
        boundary_gdf = gpd.read_file(file)

        sub_gdf = gdf_regional[gdf_regional.intersects(boundary_gdf.geometry.unary_union)].copy()

        if sub_gdf.shape[0] > 0:
            output_file = os.path.join(output_folder, gdf_regional_file+'_'+link_type+'_'+str(i+1)+'.geojson')
            print('export {} {} links from intersecting to {}'.format(sub_gdf.shape[0], link_type, output_file))
            sub_gdf.to_file(output_file,
                            driver = "GeoJSON")
        else:
            print('no link from intersecting')

In [4]:
def split_partition(df_file):
    print('------process {}'.format(df_file))
    
    df = gpd.read_file(os.path.join(data_folder, df_file+'.geojson'))
    
    # first, split into 'existing' and 'proposed'
    
    df_existing = df.loc[df.ex_class.notnull()]
    df_proposed = df.loc[df.ex_class.isnull()]
    print('Dataset {} contains {} existing links and {} proposed links'.format(
        df_file, df_existing.shape[0], df_proposed.shape[0]))
    
    # for non-regional network, export directly
    if df_file not in ['batc_bike_network_v2_epsg4326',
                       'caltrans_d4_bike_network_epsg4326',
                       'vta_bike_network_v2_epsg4326']:
        
        if df_existing.shape[0] > 0:
            print('------export existing links to {}'.format(existing_folder))
            df_existing.to_file(os.path.join(existing_folder, df_file+'_exst.geojson'),
                                driver = "GeoJSON")
        if df_proposed.shape[0] > 0:
            print('------export proposed links to {}'.format(proposed_folder))
            df_proposed.to_file(os.path.join(proposed_folder, df_file+'_ppsd.geojson'),
                                driver = "GeoJSON")
        
    # partition regional datasets
    if df_file in ['batc_bike_network_v2_epsg4326',
                   'caltrans_d4_bike_network_epsg4326',
                   'vta_bike_network_v2_epsg4326']:
        
        print('------partition existing links')
        data_partition(df_existing, df_file, 'exst', existing_folder)
        
        print('------partition proposed links')
        data_partition(df_proposed, df_file, 'ppsd', proposed_folder)

In [5]:
for i in data_list:
    split_partition(i)

------process actc_bike_network_epsg4326
Dataset actc_bike_network_epsg4326 contains 18793 existing links and 11292 proposed links
------export existing links to C:\Users\ywang\Documents\GitHub\Spatial-Analysis-Mapping-Projects\Project-Documentation\Active-Transportation-Plan-Data-Development\data\geojson\existing
------export proposed links to C:\Users\ywang\Documents\GitHub\Spatial-Analysis-Mapping-Projects\Project-Documentation\Active-Transportation-Plan-Data-Development\data\geojson\proposed
------process batc_bike_network_v2_epsg4326
Dataset batc_bike_network_v2_epsg4326 contains 2261 existing links and 730 proposed links
------partition existing links
use boundary_1 to intersect
export 66 exst links from intersecting to C:\Users\ywang\Documents\GitHub\Spatial-Analysis-Mapping-Projects\Project-Documentation\Active-Transportation-Plan-Data-Development\data\geojson\existing\batc_bike_network_v2_epsg4326_exst_1.geojson
use boundary_2 to intersect
export 243 exst links from intersecti

Dataset caltrans_d4_bike_network_epsg4326 contains 0 existing links and 144 proposed links
------partition existing links
use boundary_1 to intersect
no link from intersecting
use boundary_2 to intersect
no link from intersecting
use boundary_3 to intersect
no link from intersecting
use boundary_4 to intersect
no link from intersecting
use boundary_5 to intersect
no link from intersecting
use boundary_6 to intersect
no link from intersecting
use boundary_7 to intersect
no link from intersecting
use boundary_8 to intersect
no link from intersecting
use boundary_9 to intersect
no link from intersecting
use boundary_10 to intersect
no link from intersecting
use boundary_11 to intersect
no link from intersecting
use boundary_12 to intersect
no link from intersecting
use boundary_13 to intersect
no link from intersecting
use boundary_14 to intersect
no link from intersecting
------partition proposed links
use boundary_1 to intersect
export 3 ppsd links from intersecting to C:\Users\ywang\Do

------export proposed links to C:\Users\ywang\Documents\GitHub\Spatial-Analysis-Mapping-Projects\Project-Documentation\Active-Transportation-Plan-Data-Development\data\geojson\proposed
------process tam_bike_network_epsg4326
Dataset tam_bike_network_epsg4326 contains 832 existing links and 296 proposed links
------export existing links to C:\Users\ywang\Documents\GitHub\Spatial-Analysis-Mapping-Projects\Project-Documentation\Active-Transportation-Plan-Data-Development\data\geojson\existing
------export proposed links to C:\Users\ywang\Documents\GitHub\Spatial-Analysis-Mapping-Projects\Project-Documentation\Active-Transportation-Plan-Data-Development\data\geojson\proposed
------process vta_bike_network_v2_epsg4326
Dataset vta_bike_network_v2_epsg4326 contains 12295 existing links and 9105 proposed links
------partition existing links
use boundary_1 to intersect
no link from intersecting
use boundary_2 to intersect
no link from intersecting
use boundary_3 to intersect
no link from inters